## Flattening a nested linked list

Suppose you have a linked list where the value of each node is a sorted linked list (i.e., it is a _nested_ list). Your task is to _flatten_ this nested list—that is, to combine all nested lists into a single (sorted) linked list.

First, we'll need some code for generating nodes and a linked list:

In [89]:
# Use this class as the nodes in your linked list
class Node:
    def __init__(self, value):
        self.value = value
        self.next = None

    def __repr__(self):
        return str(self.value)


class LinkedList:
    def __init__(self, head):
        self.head = head

    def append(self, value):
        if self.head is None:
            self.head = Node(value)
            return
        node = self.head
        while node.next is not None:
            node = node.next
        node.next = Node(value)

    def flatten_deep(self):
        value_list = []
        node = self.head

        while node.next is not None:
            value_list.append(node.value)
            node = node.next
        value_list.append(node.value)  # append the last value after exiting out of the loop
        return value_list
    
    # custom-defined by me      
    def __eq__(self, other):
        ix = 0
        node = self.head
        while node.next is not None:
            if node.value == other[ix]:
                node = node.next
                ix += 1
            else:
                return False
        return True
    
    def __repr__(self):
        res = []
        node = self.head
        while node:
            res.append(node.value)
            node = node.next
        return res.__repr__()

Now, in the cell below, see if you can solve the problem by implementing the `flatten` method.

>Hint: If you first create a `merge` method that merges two linked lists into a sorted linked list, then there is an elegant recursive solution.

In [93]:
class NestedLinkedList(LinkedList):
    
    @staticmethod
    def merge(a, b):

        if a is None:
            return b
        elif b is None:
            return a
        
        merged = LinkedList(None)
        
        na = a.head
        nb = b.head
        
        while na is not None or nb is not None:
            
            # if one list is empty, keep appending the 2nd list node by node
            if na is None:
                merged.append(nb.value)  # original sol. appends the Node itself which is wrong
                nb = nb.next
            elif nb is None:
                merged.append(na.value)
                na = na.next
                
            # if both are non-empty, compare 2 values and append the smaller one
            elif na.value <= nb.value:
                merged.append(na.value)
                na = na.next
            else:
                merged.append(nb.value)
                nb = nb.next
                
        return merged

    def flatten(self):
        return self._flatten(self.head)
    
    def _flatten(self, node):
        if node.next is not None:
            return self.merge(node.value, self._flatten(node.next))
        else:
            return self.merge(node.value, None)


Here's some code that will generate a nested linked list that we can use to test the solution:

In [94]:
# Setup for testing
linked_list = LinkedList(Node(1))
linked_list.append(3)
linked_list.append(5)

second_linked_list = LinkedList(Node(2))
second_linked_list.append(4)

nested_linked_list = NestedLinkedList(Node(linked_list))
nested_linked_list.append(second_linked_list)

### Structure
`nested_linked_list` should now have 2 nodes.  The head node is a linked list containing `1, 3, 5`.  The second node is a linked list containing `2, 4`.

Calling `flatten` should return a linked list containing `1, 2, 3, 4, 5`.

In [95]:
solution = nested_linked_list.flatten()
print(solution)
assert solution == [1,2,3,4,5]

[1, 2, 3, 4, 5]


### Computational Complexity
Lets start with the computational complexity of `merge`.  Merge takes in two lists.  Let's say the lengths of the lists are $N_{1}$ and $N_{2}$. Because we assume the inputs are sorted, `merge` is very efficient. It looks at the first element of each list and adds the smaller one to the returned list.  Every time through the loop we are appending one element to the list, so it will take $N_{1} + N_{2}$ iterations until we have the whole list.

The complexity of `flatten` is a little more complicated to calculate.  Suppose our `NestedLinkedList` has $N$ linked lists and each list's length is represented by $M_{1}, M_{2}, ..., M_{N}$.

We can represent this recursion as:

$merge(M_{1}, merge(M_{2}, merge(..., merge(M_{N-1}, merge(M_{N}, None)))))$

Let's start from the inside.  The inner most merge returns the $nth$ linked list.  The next merge does $M_{N-1} + M_{N}$ comparisons.  The next merge does $M_{N-2} + M_{N-1} + M_{N}$ comparisons.

Eventually we will do $N$ comparisons on all of the $M_{N}$ elements. We will do $N-1$ comparisons on $M_{N-1}$ elements.

This can be generalized as:

$$
\sum_n^N n*M_{n}
$$